# Experimental Comparison Between FastGradRidgeLogit And Sklearn

## About Data

#### Data Name:  

SPAM E-mail Data

#### Description:

Number of Instances: 4601 (1813 Spam = 39.4%)

Number of Attributes: 58 (57 continuous, 1 nominal class label)

The last column of 'spambase.data' denotes whether the e-mail was 
considered spam (1) or not (0)

To explore full documentation of the data set, please check [https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.info.txt](https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.info.txt)

## Data Process Before Model Training


In [1]:
import pandas as pd
import numpy as np
import sklearn.preprocessing

spam = pd.read_table('https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.data', sep=' ', header=None)
test_indicator = pd.read_table('https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/spam.traintest', sep=' ',
                               header=None)

x = np.asarray(spam)[:, 0:-1]
y = np.asarray(spam)[:, -1]*2 - 1  # Convert to +/- 1
test_indicator = np.array(test_indicator).T[0]

# Divide the data into train, test sets
x_train = x[test_indicator == 0, :]
x_test = x[test_indicator == 1, :]
y_train = y[test_indicator == 0]
y_test = y[test_indicator == 1]

# Standardize the data. 
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Train Model Using FastGradRidgeLogit

In [2]:
from fgrlogit import FastGradRidgeLogit

fg = FastGradRidgeLogit()
fg.fit(lambduh = 0.1,x = x_train,y = y_train)

Start fast gradient descent:
Fast gradient iteration 100
Fast gradient iteration 200
Fast gradient iteration 300


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00811784, -0.00290854,  0.01525627, ...,  0.00797122,
         0.01367347,  0.01683449],
       [ 0.01430038, -0.00561651,  0.02786037, ...,  0.01446999,
         0.02484187,  0.0308083 ],
       ..., 
       [ 0.02117346, -0.03877587,  0.0977377 , ...,  0.06871946,
         0.12509328,  0.14818874],
       [ 0.02117345, -0.03877587,  0.09773769, ...,  0.06871949,
         0.12509325,  0.14818874],
       [ 0.02117345, -0.03877586,  0.09773769, ...,  0.0687195 ,
         0.12509324,  0.14818872]])

## Compare Results With Sklearn

**Beta**

Beta values of FastGradRidgeLogit at the last iteration:

In [3]:
fg.opt_betas

array([ 0.02117345, -0.03877586,  0.09773769,  0.05503797,  0.15256407,
        0.13534363,  0.28352689,  0.15114453,  0.11280869,  0.06183421,
        0.10501987, -0.04219954,  0.03472871,  0.02859179,  0.10917878,
        0.27363769,  0.17373128,  0.1251736 ,  0.1290988 ,  0.122432  ,
        0.22405222,  0.10833272,  0.24549147,  0.16655941, -0.14754049,
       -0.10493027, -0.11498963, -0.06087832, -0.04741804, -0.0662691 ,
       -0.02882516, -0.01515314, -0.07621362, -0.01594241, -0.04245242,
       -0.01357186, -0.0770049 , -0.03415872, -0.07954049,  0.01733621,
       -0.0477297 , -0.09082765, -0.06004328, -0.06969741, -0.11271631,
       -0.11263901, -0.03213438, -0.06206713, -0.05727233, -0.04253644,
       -0.02823828,  0.15484326,  0.26205836,  0.058939  ,  0.0687195 ,
        0.12509324,  0.14818872])

Sklearn result:

In [7]:
import sklearn.linear_model

lambduh = 0.1
n_train = len(y_train)

lr = sklearn.linear_model.LogisticRegression(penalty='l2', C=1/(2*lambduh*n_train), fit_intercept=False, tol=10e-8, max_iter=1000)
lr.fit(x_train, y_train)
print(lr.coef_)

[[ 0.02117345 -0.03877587  0.09773771  0.05503819  0.15256401  0.13534366
   0.28352681  0.15114461  0.11280873  0.06183421  0.10502022 -0.04219962
   0.03472887  0.02859174  0.10917878  0.27363777  0.17373109  0.12517347
   0.1290986   0.12243222  0.2240522   0.10833277  0.24549208  0.16655949
  -0.14754037 -0.10493049 -0.1149898  -0.06087818 -0.04741795 -0.06626914
  -0.02882511 -0.0151532  -0.07621369 -0.0159425  -0.04245255 -0.01357162
  -0.07700499 -0.03415877 -0.07954035  0.01733619 -0.04772964 -0.09082759
  -0.06004341 -0.06969745 -0.11271606 -0.11263913 -0.0321344  -0.06206727
  -0.05727232 -0.0425365  -0.02823825  0.15484338  0.26205747  0.05893876
   0.06871901  0.12509356  0.14818851]]


**Objective value:**

Objective value of FastGradRidgeLogit:

In [8]:
fg.objective(beta = fg.opt_betas, lambduh = 0.1, x = x_train, y = y_train)

0.44122267806605076

Objective value of sklearn:

In [9]:
fg.objective(beta = lr.coef_.flatten(), lambduh = 0.1, x = x_train, y = y_train)

0.441222678065749